In [1]:
import os
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

from llama_index.core import Settings

In [2]:
docs = SimpleDirectoryReader(input_dir="./data").load_data()

In [3]:
docs[0].__dict__

{'id_': '55a7bd97-d90d-4628-9715-a45faa5811bb',
 'embedding': None,
 'metadata': {'file_path': '/mnt/d/DEV/MOC-Training/data/sample_doc_thai_1.txt',
  'file_name': 'sample_doc_thai_1.txt',
  'file_type': 'text/plain',
  'file_size': 955,
  'creation_date': '2025-07-15',
  'last_modified_date': '2025-07-15'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': MediaResource(embeddings=None, data=None, text='ปัญญาประดิษฐ์ (AI) ในประเทศไทยกำลังพัฒนาอย่างรวดเร็ว หน่วยงานทั้งภาครัฐและเอกชนให้ความสำคัญกับการนำ AI ไปประยุกต์ใช้ในหลายด้าน เช่น การแพทย์ การศึกษา การเกษตร และอุตสาหกรรม\n\nหนึ่งในหัวข้อที่ได้รับความสนใจมากคือ การนำโมเดลภาษา (LLM) มาช่วยในการ

In [4]:

import faiss

# dimensions of BAAI/bge-m3
d = 1024
faiss_index = faiss.IndexFlatIP(d)

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


In [6]:
embedding_model_name = 'BAAI/bge-m3'
# embedding_model_name = 'kornwtp/simcse-model-phayathaibert'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024, device='cpu')

In [7]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# ตั้งค่าให้แยกเอกสารเป็น chunk ย่อย ๆ
text_splitter = SentenceSplitter(
    separator=" ",       # ใช้เว้นวรรคเป็นตัวแยก
    chunk_size=128,     # ขนาดของแต่ละ chunk (จำนวนตัวอักษร)
    chunk_overlap=10    # ความซ้อนของข้อความระหว่างแต่ละ chunk
)

# สร้าง ingestion pipeline ที่ประกอบด้วย text_splitter
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
    ]
)

# รัน pipeline กับเอกสารที่โหลดไว้ เพื่อแยกเป็น chunks
nodes = pipeline.run(
    documents=docs,
    in_place=True,       # แก้ไขเอกสารเดิมแทนที่จะสร้างใหม่
    show_progress=True,  # แสดง progress bar ขณะรัน
)


Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


In [9]:
# เริ่มต้น client ของ ChromaDB และสร้าง collection สำหรับเก็บเวกเตอร์
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("rag_demo")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# สร้าง storage context สำหรับบันทึกข้อมูล
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# สร้าง index จากเอกสาร โดยใช้ embedding ที่เรากำหนด
index = VectorStoreIndex(
    nodes=nodes, storage_context=storage_context, embed_model=embed_model,show_progress=True,  # แสดง progress bar ขณะรัน
)

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
index.storage_context.persist(persist_dir="./storage/")

In [13]:
retriever = index.as_retriever(similarity_top_k=5)
response = retriever.retrieve('LlamaIndex ')
for i in range(5):
    print('Response:', response[i].text,'\n')
    print('Score:', response[i].score)
    print('-'*50,'\n\n')

Response: LlamaIndex เป็นเครื่องมือที่ช่วยเชื่อมโยงโมเดลภาษาเข้ากับฐานข้อมูลหรือเอกสารได้อย่างง่ายดาย จุดเด่นคือรองรับการสกัด metadata เช่น ชื่อเรื่อง 

Score: 0.48732581252309376
-------------------------------------------------- 


Response: embedding ที่นิยมใช้กับภาษาไทยในปัจจุบัน เช่น BAAI/bge-m3 สามารถนำมาใช้ร่วมกับ LlamaIndex เพื่อสร้างระบบ RAG ที่รองรับภาษาไทยได้ดี 

Score: 0.39552324242833836
-------------------------------------------------- 


Response: การนำโมเดลภาษา (LLM) มาช่วยในการวิเคราะห์และสรุปข้อมูลภาษาไทย 

Score: 0.34207653990496506
-------------------------------------------------- 


Response: ชื่อเรื่อง คำสำคัญ สาระสังเขป ฯลฯ เพื่อช่วยให้การค้นหาข้อมูลมีประสิทธิภาพมากขึ้น

โมเดล embedding 

Score: 0.29747283041860656
-------------------------------------------------- 


Response: เพราะส่งผลต่อความแม่นยำในการค้นคืนข้อมูล หาก chunk เล็กเกินไปอาจขาดบริบท หากใหญ่เกินไปอาจทำให้ค้นหาข้อมูลยากขึ้น 

Score: 0.289866336458224
----------------------------------------------

In [11]:
from llama_index.core import load_index_from_storage

In [16]:
CHROMA_INDEX_PATH = "./chroma_db/"
persist_dir = "./storage/"
chroma_client = chromadb.PersistentClient(path=CHROMA_INDEX_PATH)
chroma_collection = chroma_client.get_or_create_collection("rag_demo")

# Create vector store
chroma_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_ctx = StorageContext.from_defaults(vector_store=chroma_store, persist_dir=persist_dir)
index = load_index_from_storage(storage_context=storage_ctx)
engine = index.as_query_engine(similarity_top_k=5)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/index_store.json.


In [17]:
query = "AI สำคัญกับประเทศไทยอย่างไร"
response = engine.query(query)
print(response)

Context information is below.
---------------------
file_path: /mnt/d/DEV/MOC-Training/data/sample_doc_thai_1.txt

ปัญญาประดิษฐ์ (AI) ในประเทศไทยกำลังพัฒนาอย่างรวดเร็ว หน่วยงานทั้งภาครัฐและเอกชนให้ความสำคัญกับการนำ AI

file_path: /mnt/d/DEV/MOC-Training/data/sample_doc_thai_1.txt

AI ไปประยุกต์ใช้ในหลายด้าน เช่น การแพทย์ การศึกษา การเกษตร และอุตสาหกรรม

หนึ่งในหัวข้อที่ได้รับความสนใจมากคือ

file_path: /mnt/d/DEV/MOC-Training/data/sample_doc_thai_3.txt

embedding ที่นิยมใช้กับภาษาไทยในปัจจุบัน เช่น BAAI/bge-m3 สามารถนำมาใช้ร่วมกับ LlamaIndex เพื่อสร้างระบบ RAG ที่รองรับภาษาไทยได้ดี

file_path: /mnt/d/DEV/MOC-Training/data/sample_doc_thai_2.txt

ระบบ Retrieval-Augmented Generation (RAG) เป็นเทคนิคที่ช่วยให้โมเดลภาษาใหญ่สามารถตอบคำถามจากข้อมูลที่อยู่นอกเหนือจากที่ใช้ฝึกอบรม

file_path: /mnt/d/DEV/MOC-Training/data/sample_doc_thai_2.txt

ตัวอย่างการนำไปใช้ เช่น การตอบคำถามจากเอกสารหรือฐานความรู้ขององค์กร

การเลือกขนาด chunk ที่เหมาะสมเป็นปัจจัยสำคัญ
---------------------
Given the context 